In [1]:
from edflib import EDFwriter
import mne
import os
from pathlib import Path
import numpy as np
import math

In [2]:
def run_conversion(raw, raw_fpath, outdir):
    ch_names = raw.info['ch_names']
    n_chan = len(ch_names)
    sfreq = int(raw.info['sfreq'])
    n_times = raw.n_times
    n_secs = n_times / sfreq
    raw_name = raw_fpath.name
    out_name = raw_name.replace(".set", ".edf")
    out_fpath = Path(outdir) / out_name
    hdl = EDFwriter(out_fpath, EDFwriter.EDFLIB_FILETYPE_EDFPLUS, n_chan)
    set_channel_info(hdl, ch_names, sfreq)
    set_patient_info(hdl)
    write_data(raw, hdl, sfreq, n_secs, n_times)
    hdl.close()

In [3]:
def set_channel_info(hdl, ch_names, sfreq):
    for chan, ch in enumerate(ch_names):
        if hdl.setPhysicalMaximum(chan, 3000) != 0:
            print("setPhysicalMaximum() returned an error")
            sys.exit()
        if hdl.setPhysicalMinimum(chan, -3000) != 0:
            print("setPhysicalMinimum() returned an error")
            sys.exit()
        if hdl.setDigitalMaximum(chan, 32767) != 0:
            print("setDigitalMaximum() returned an error")
            sys.exit()
        if hdl.setDigitalMinimum(chan, -32768) != 0:
            print("setDigitalMinimum() returned an error")
            sys.exit()
        if hdl.setPhysicalDimension(chan, "uV") != 0:
            print("setPhysicalDimension() returned an error")
            sys.exit()
        if hdl.setSampleFrequency(chan, sfreq) != 0:
            print("setSampleFrequency() returned an error")
            sys.exit()
        if hdl.setSignalLabel(chan, ch) != 0:
            print("setSignalLabel() returned an error")
            sys.exit()
        if hdl.setPreFilter(chan, "HP:0.05Hz LP:40Hz N:60Hz") != 0:
            print("setPreFilter() returned an error")
            sys.exit()
        if hdl.setTransducer(chan, "AgAgCl cup electrode") != 0:
            print("setTransducer() returned an error")
            sys.exit()


In [4]:
def set_patient_info(hdl):
    if hdl.setPatientCode("1234567890") != 0:
        print("setPatientCode() returned an error")
        sys.exit()
    if hdl.setPatientBirthDate(1913, 4, 7) != 0:
        print("setPatientBirthDate() returned an error")
        sys.exit()
    if hdl.setPatientName("Smith J.") != 0:
        print("setPatientName() returned an error")
        sys.exit()
    if hdl.setAdditionalPatientInfo("normal condition") != 0:
        print("setAdditionalPatientInfo() returned an error")
        sys.exit()
    if hdl.setAdministrationCode("1234567890") != 0:
        print("setAdministrationCode() returned an error")
        sys.exit()
    if hdl.setTechnician("Black Jack") != 0:
        print("setTechnician() returned an error")
        sys.exit()
    if hdl.setEquipment("recorder") != 0:
        print("setEquipment() returned an error")
        sys.exit()
    if hdl.setAdditionalRecordingInfo("nothing special") != 0:
        print("setAdditionalRecordingInfo() returned an error")
        sys.exit()

In [5]:
def write_data(raw, hdl, sfreq, n_secs, n_times):
    data = raw.get_data()
    for ch in range(data.shape[0]):
        buf = np.empty(sfreq, np.float64, "C")
        ch_data = data[ch,:]

        for s in range(math.ceil(n_secs)):
            end_samp = (s+1)*sfreq
            if end_samp > n_times:
                end_samp = n_times
            chunk = ch_data[s*sfreq:end_samp]
            for ind,c in enumerate(chunk):
                buf[ind] = c*1000000  # convert to microvolts
            err = hdl.writeSamples(buf)
            if err != 0:
                print("writeSamples() returned error: %d" %(err))
                break;

In [6]:
sourcedir = Path("D:/ScalpData/edf_write_validation/sourcedata")
eeglab_fpaths = sourcedir.glob("*.set")
outdir = Path("D:/ScalpData/edf_write_validation/derivatives/edf")
for fpath in eeglab_fpaths:
    raw = mne.io.read_raw_eeglab(fpath)
    run_conversion(raw, fpath, outdir)

Reading D:\ScalpData\edf_write_validation\sourcedata\001.fdt
Reading D:\ScalpData\edf_write_validation\sourcedata\002.fdt
Reading D:\ScalpData\edf_write_validation\sourcedata\003.fdt
Reading D:\ScalpData\edf_write_validation\sourcedata\004.fdt
Reading D:\ScalpData\edf_write_validation\sourcedata\005.fdt
Reading D:\ScalpData\edf_write_validation\sourcedata\006.fdt
Reading D:\ScalpData\edf_write_validation\sourcedata\007.fdt
Reading D:\ScalpData\edf_write_validation\sourcedata\008.fdt
Reading D:\ScalpData\edf_write_validation\sourcedata\009.fdt
Reading D:\ScalpData\edf_write_validation\sourcedata\010.fdt
Reading D:\ScalpData\edf_write_validation\sourcedata\011.fdt
Reading D:\ScalpData\edf_write_validation\sourcedata\012.fdt
Reading D:\ScalpData\edf_write_validation\sourcedata\013.fdt
Reading D:\ScalpData\edf_write_validation\sourcedata\014.fdt
Reading D:\ScalpData\edf_write_validation\sourcedata\015.fdt
Reading D:\ScalpData\edf_write_validation\sourcedata\016.fdt
Reading D:\ScalpData\edf